<a href="https://colab.research.google.com/github/gc2321/apache_beam/blob/main/3_side_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!{'pip install --quiet apache_beam'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup

In [3]:
#!{'pip install --quiet apache-beam[interactive]'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.3/406.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.5/816.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.4 which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.24.0 which is incompatible.


In [6]:
import apache_beam as beam

## Advance Combine

In [9]:
class AverageFn(beam.CombineFn):

  def create_accumulator(self):
     return (0.0, 0)   # initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):

    ind_sums, ind_counts = zip(*accumulators)       # zip - [(27, 3), (39, 3), (18, 2)]  -->   [(27,39,18), (3,3,2)]
    return sum(ind_sums), sum(ind_counts)        # (84,8)

  def extract_output(self, sum_count):

    (sum, count) = sum_count    # combine globally using CombineFn
    return sum / count if count else float('NaN')

p = beam.Pipeline()
small_sum = (
    p
    | beam.Create([15,5,7,7,9,23,13,5])
    | "Combine Globally" >> beam.CombineGlobally(AverageFn())
    | "Write Result" >> beam.io.WriteToText('data/combine')
)

p.run()

In [11]:
!head -n 20 data/combine-00000-of-00001

10.5


## Composite Transforms

In [12]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
import fileinput
filePath ="/content/gdrive/MyDrive/beam/dept_data.txt"

In [14]:
class DeptTransform(beam.PTransform):
  def expand(self, input_coll):
    a = (
        input_coll
        | 'Group and Combine' >> beam.CombinePerKey(sum)
        | 'Count Filter Accounts' >> beam.Filter(filter_on_count)
        | 'Regular accounts employee' >> beam.Map(format_output)
    )
    return a

def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

def format_output(element):
  name, count = element
  #return ', '.join((name.encode('ascii'),str(count),'Regular employee'))
  return ', '.join((name,str(count),'Regular employee'))

# ParDo
class SplitRow(beam.DoFn):
  def process(self, element):
    return [element.split(',')]

class FilterAccountsEmployee(beam.DoFn):
  def __init__(self, acc):
        self.acc = acc

  def process(self, element):
    if element[3] == self.acc:
      return [element]

class PairEmployee(beam.DoFn):
  def process(self, element):
    return [(element[3] + "," + element[1], 1)]

class Counting(beam.DoFn):
  def process(self, element):
    (key, values) = element
    return [(key, sum(values))]

In [18]:
p = beam.Pipeline()


input_collection = (
                      p
                      | "Read from text file" >> beam.io.ReadFromText(filePath)
                      | "Split rows" >> beam.ParDo(SplitRow())
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.ParDo(FilterAccountsEmployee('Accounts'))
                      | 'Pair each accounts employee with 1' >> beam.ParDo(PairEmployee())
                      | 'Composite Transform ACC' >> DeptTransform()
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.ParDo(FilterAccountsEmployee('HR'))
                | 'Pair each hr employee with 1' >> beam.ParDo(PairEmployee())
                | 'Composite Transform HR' >> DeptTransform()
           )

output =(
         (accounts_count,hr_count)
          | beam.Flatten()
          | beam.io.WriteToText('data/both')
)

p.run()

In [19]:
!head -n 20 data/both-00000-of-00001

Accounts,Marco, 31, Regular employee
Accounts,Rebekah, 31, Regular employee
Accounts,Itoe, 31, Regular employee
Accounts,Edouard, 31, Regular employee
Accounts,Kyle, 62, Regular employee
Accounts,Kumiko, 31, Regular employee
Accounts,Gaston, 31, Regular employee
HR,Beryl, 62, Regular employee
HR,Olga, 31, Regular employee
HR,Leslie, 31, Regular employee
HR,Mindy, 31, Regular employee
HR,Vicky, 31, Regular employee
HR,Richard, 31, Regular employee
HR,Kirk, 31, Regular employee
HR,Kaori, 31, Regular employee
HR,Oscar, 31, Regular employee


## Co-Group By Key

In [20]:
deptData ="/content/gdrive/MyDrive/beam/dept_data.txt"
locationData = "/content/gdrive/MyDrive/beam/location.txt"

In [21]:
p1 = beam.Pipeline()

def retTuple(element):
  thisTuple=element.split(',')
  return (thisTuple[0], thisTuple[1:])

# Apply a ParDo to the PCollection "words" to compute lengths for each word.
dep_rows = (
                p1
                | "Reading File 1" >> beam.io.ReadFromText(deptData)
                | 'Pair each employee with key' >> beam.Map(retTuple)          # {149633CM : [Marco,10,Accounts,1-01-2019]}
               )


loc_rows = (
                p1
                | "Reading File 2" >> beam.io.ReadFromText(locationData)
                | 'Pair each loc with key' >> beam.Map(retTuple)                # {149633CM : [9876843261,New York]}
               )


results = ({'dep_data': dep_rows, 'loc_data': loc_rows}
           | beam.CoGroupByKey()
           | 'Write results' >> beam.io.WriteToText('data/result')
          )


p1.run()

!{('head -n 20 data/result-00000-of-00001')}

('149633CM', {'dep_data': [['Marco', '10', 'Accounts', '1-01-2019'], ['Marco', '10', 'Accounts', '2-01-2019'], ['Marco', '10', 'Accounts', '3-01-2019'], ['Marco', '10', 'Accounts', '4-01-2019'], ['Marco', '10', 'Accounts', '5-01-2019'], ['Marco', '10', 'Accounts', '6-01-2019'], ['Marco', '10', 'Accounts', '7-01-2019'], ['Marco', '10', 'Accounts', '8-01-2019'], ['Marco', '10', 'Accounts', '9-01-2019'], ['Marco', '10', 'Accounts', '10-01-2019'], ['Marco', '10', 'Accounts', '11-01-2019'], ['Marco', '10', 'Accounts', '12-01-2019'], ['Marco', '10', 'Accounts', '13-01-2019'], ['Marco', '10', 'Accounts', '14-01-2019'], ['Marco', '10', 'Accounts', '15-01-2019'], ['Marco', '10', 'Accounts', '16-01-2019'], ['Marco', '10', 'Accounts', '17-01-2019'], ['Marco', '10', 'Accounts', '18-01-2019'], ['Marco', '10', 'Accounts', '19-01-2019'], ['Marco', '10', 'Accounts', '20-01-2019'], ['Marco', '10', 'Accounts', '21-01-2019'], ['Marco', '10', 'Accounts', '22-01-2019'], ['Marco', '10', 'Accounts', '23-01-2